<a href="https://colab.research.google.com/github/luqmanzaceria/karpathy/blob/main/micrograd_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [51]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [52]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  da = -3*a**2 - 0.5*a**-0.5
  db = 3*cos(3*b) + 2.5*b**1.5
  dc = 1/c**2

  return [da, db, dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [53]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
numerical_grad = [(f(2.000001,1,1)-f(2,1,1))/0.000001, (f(1,3.000001,1)-f(1,3,1))/0.000001, (f(1,1,4.000001)-f(1,1,4))/0.000001] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984299080324


In [54]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [(f(2.000001,1,1)-f(1.999999,1,1))/0.000002, (f(1,3.000001,1)-f(1,2.999999,1))/0.000002, (f(1,1,4.000001)-f(1,1,3.999999))/0.000002] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390909155
OK for dim 1: expected 10.25699027111255, yours returns 10.256990273127542
OK for dim 2: expected 0.0625, yours returns 0.06250000006424727


## section 2: support for softmax

In [75]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------

  def exp(self):
    out = Value(exp(self.data), (self,), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def log(self):
    out = Value(log(self.data), (self,), 'log')

    def _backward():
      self.grad += out.grad * (1/self.data)
    out._backward = _backward

    return out

  def __radd__(self, other):  # to handle int + Value and float + Value
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out
  
  def __truediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data**-1, (self, other), '/')

    def _backward():
      self.grad += out.grad * (1/other.data)
      other.grad += out.grad * (-self.data/(other.data**2))
    out._backward = _backward

    return out
  
  def __pow__(self, other):
    out = Value(self.data ** other, (self, other), '**')
    
    def _backward():
      self.grad += out.grad * (other * (self.data ** (other - 1)))
    out._backward = _backward

    return out
  
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += out.grad * other
      other.grad += out.grad * self.data
    out._backward = _backward

    return out 
  
  def __sub__(self, other):
    out = Value(Value(self.data) + (-other), (self, other), '-')

    def _backward():
      self.grad += out.grad
      other.grad += -out.grad

    out._backward = _backward
    return out
  
  def __neg__(self):
    out = Value(-self.data, (self,), '-')

    def _backward():
      self.grad += -out.grad
    out._backward = _backward

    return out

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [76]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535044
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625318
OK for dim 2: expected 0.005653302662216329, yours returns 0.0056533026622163285
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [94]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

# Define the logits using your custom Value class
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log()  # dim 3 acts as the label for this input example
loss.backward()

# Display the gradients calculated by your Value class
print("Gradients from custom Value class:")
for i, logit in enumerate(logits):
    print(f"logit {i} grad: {logit.grad}")

# Now, do the same computation with torch

# Define the logits using torch tensors
logits_torch = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)
probs_torch = torch.softmax(logits_torch, dim=0)
loss_torch = -torch.log(probs_torch[3])  # dim 3 acts as the label for this input example

# Compute the gradients using torch
loss_torch.backward()

# Display the gradients calculated by torch
print("\nGradients from torch:")
for i, grad in enumerate(logits_torch.grad):
    print(f"logit {i} grad: {grad.item()}")

# Compare the gradients
print("\nComparison of gradients:")
for i in range(4):
    custom_grad = logits[i].grad
    torch_grad = logits_torch.grad[i].item() if logits_torch.grad[i] is not None else None
    match = abs(custom_grad - torch_grad) < 1e-5 if torch_grad is not None else False
    print(f"logit {i}: {'OK' if match else 'WRONG!'} | custom: {custom_grad}, torch: {torch_grad}")

Gradients from custom Value class:
logit 0 grad: 0.04177257051535044
logit 1 grad: 0.8390245074625318
logit 2 grad: 0.0056533026622163285
logit 3 grad: -0.8864503806400986

Gradients from torch:
logit 0 grad: 0.041772566735744476
logit 1 grad: 0.8390244841575623
logit 2 grad: 0.005653302650898695
logit 3 grad: -0.8864504098892212

Comparison of gradients:
logit 0: OK | custom: 0.04177257051535044, torch: 0.041772566735744476
logit 1: OK | custom: 0.8390245074625318, torch: 0.8390244841575623
logit 2: OK | custom: 0.0056533026622163285, torch: 0.005653302650898695
logit 3: OK | custom: -0.8864503806400986, torch: -0.8864504098892212
